In [1]:
debug = False

___
___
# **Librairies**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import VotingRegressor

import xgboost as xgb
import lightgbm as lgb
import catboost
from lightgbm.sklearn import LGBMClassifier
from catboost import CatBoostClassifier
from tqdm import tqdm
import gc
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

___
___
# **Open Data**

In [3]:
df       = pd.read_csv('/kaggle/input/trustii-data/train.csv')
test_df  = pd.read_csv('/kaggle/input/trustii-data/test.csv')

if 'trustii_id' in df :
    df = df.drop(columns = ['trustii_id'])

print(df.shape, test_df.shape)
display(df.head())

(23282, 887) (9870, 888)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattage,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_etoile,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_general,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_monoprix,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_viviers,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage de bovins,flag_Entreposage de distribution de denrées alimentaires,flag_Entreposage de produits de la pêche,flag_Entreposage de pr

In [4]:
# Création d'une colonne "Length SIRET"
df['len_SIRET'] = df['SIRET'].astype(str).str.len()
df['SIRET_14char'] = (df['len_SIRET']==14).astype(int)

In [5]:
dico_y = {'A corriger de manière urgente' : 0,
          'A améliorer' : 1,
          'Satisfaisant' : 2,
          'Très satisfaisant' : 3,
         }
dico_y_INVERSE = {v:k for k, v in dico_y.items()}

___
___
# **Open SIRET**

In [6]:
a = pd.read_csv('/kaggle/input/siret-stocketablissement/StockEtablissement_utf8.csv', nrows=5)

a.head(1)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
0,325175,16,32517500016,O,2000-09-26,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,3,NaN,NaN,NaN,NaN,MANIHI COTE MONTAGNE TUAMOTU,98770,MANIHI,NaN,NaN,98727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-05-27,F,NaN,NaN,NaN,NaN,32.12Z,NAFRev2,N


In [7]:
print(a.columns)

Index(['siren', 'nic', 'siret', 'statutDiffusionEtablissement',
       'dateCreationEtablissement', 'trancheEffectifsEtablissement',
       'anneeEffectifsEtablissement',
       'activitePrincipaleRegistreMetiersEtablissement',
       'dateDernierTraitementEtablissement', 'etablissementSiege',
       'nombrePeriodesEtablissement', 'complementAdresseEtablissement',
       'numeroVoieEtablissement', 'indiceRepetitionEtablissement',
       'typeVoieEtablissement', 'libelleVoieEtablissement',
       'codePostalEtablissement', 'libelleCommuneEtablissement',
       'libelleCommuneEtrangerEtablissement',
       'distributionSpecialeEtablissement', 'codeCommuneEtablissement',
       'codeCedexEtablissement', 'libelleCedexEtablissement',
       'codePaysEtrangerEtablissement', 'libellePaysEtrangerEtablissement',
       'complementAdresse2Etablissement', 'numeroVoie2Etablissement',
       'indiceRepetition2Etablissement', 'typeVoie2Etablissement',
       'libelleVoie2Etablissement', 'codePostal2

In [8]:
%%time

cols = ['siret',
        'dateCreationEtablissement',
        'dateDebut',
        'trancheEffectifsEtablissement',
        'activitePrincipaleRegistreMetiersEtablissement',
        'dateDernierTraitementEtablissement',
        'etablissementSiege',
        'nombrePeriodesEtablissement',
        'activitePrincipaleEtablissement',
        'caractereEmployeurEtablissement',
        
        'indiceRepetitionEtablissement',
        #'libelleVoieEtablissement',
        'codeCommuneEtablissement',
        'codeCedexEtablissement',
       ]

if debug :
    SIRET = pd.read_csv('/kaggle/input/siret-stocketablissement/StockEtablissement_utf8.csv', usecols = cols, nrows=1000000)
else :
    SIRET = pd.read_csv('/kaggle/input/siret-stocketablissement/StockEtablissement_utf8.csv', usecols = cols)
    
print(SIRET.shape)
SIRET.head()

(38195474, 13)
CPU times: user 2min 58s, sys: 43.4 s, total: 3min 42s
Wall time: 4min 48s


,siret,dateCreationEtablissement,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,dateDebut,activitePrincipaleEtablissement,caractereEmployeurEtablissement
0,32517500016,2000-09-26,NaN,3212ZZ,2015-03-18T00:58:59,False,3,NaN,98727,NaN,2009-05-27,32.12Z,N
1,32517500024,2008-05-20,NaN,NaN,2011-12-12T09:40:04,False,2,NaN,84007,NaN,2011-10-21,47.89Z,N
2,32517500032,2009-05-27,NaN,NaN,2014-07-08T00:10:21,False,2,NaN,84007,NaN,2011-10-21,32.12Z,N
3,32517500040,2011-10-21,NaN,3212ZZ,2015-03-18T00:58:59,False,2,NaN,13042,NaN,2014-01-07,32.12Z,N
4,32517500057,2014-01-07,NaN,NaN,2018-07-10T14:17:15,False,2,NaN,13204,NaN,2018-02-07,32.12Z,N


In [9]:
SIRET['codeCedexEtablissement_2char']   = SIRET['codeCedexEtablissement'].astype(str).apply(lambda x : x[:2])
SIRET['codeCommuneEtablissement_2char'] = SIRET['codeCommuneEtablissement'].astype(str).apply(lambda x : x[:2])

In [10]:
SIRET['siret'].astype(str).str.len().value_counts()

siret
14    37995414
13      190797
12        9257
11           6
Name: count, dtype: int64

___
___
# **Jointure**

In [11]:
SIRET.head(2)

,siret,dateCreationEtablissement,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,dateDebut,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char
0,32517500016,2000-09-26,NaN,3212ZZ,2015-03-18T00:58:59,False,3,NaN,98727,NaN,2009-05-27,32.12Z,N,na,98
1,32517500024,2008-05-20,NaN,NaN,2011-12-12T09:40:04,False,2,NaN,84007,NaN,2011-10-21,47.89Z,N,na,84


In [12]:
info_SIRET = pd.concat([
                        df[['SIRET', 'Date_inspection', 'Numero_inspection']],
                        test_df[['SIRET', 'Date_inspection', 'Numero_inspection']],
                       ]
                      ).drop_duplicates().reset_index(drop=True)

# Aperçu
print(info_SIRET.shape)
info_SIRET.head(10)

(32388, 3)


,SIRET,Date_inspection,Numero_inspection
0,50846842800034,2024-02-05 00:00:00+00:00,23-110093-1
1,90472398800013,2023-07-17 00:00:00+00:00,23-056603-1
2,48792443300013,2023-03-08 00:00:00+00:00,23-018114-1
3,80481822700022,2023-02-15 00:00:00+00:00,23-011747-1
4,53444598600019,2023-11-21 00:00:00+00:00,23-097885-1
5,32396242300022,2023-02-17 00:00:00+00:00,23-013970-1
6,44787745700013,2023-08-07 00:00:00+00:00,23-067226-1
7,82793881200019,2023-09-25 00:00:00+00:00,23-076581-1
8,81372822700014,2023-08-09 00:00:00+00:00,23-063340-1
9,84386286300019,2023-03-31 00:00:00+00:00,23-031219-1


In [13]:
%%time

# Passage en str pour la jointure
df['SIRET'] = df['SIRET'].astype(str)
SIRET['siret'] = SIRET['siret'].astype(str)

CPU times: user 21.7 s, sys: 4.25 s, total: 25.9 s
Wall time: 25.8 s


In [14]:
%%time

# Jointure
info_SIRET = pd.merge(info_SIRET,
                      SIRET,
                      how='left',
                      left_on='SIRET',
                      right_on='siret',
                     )

# Aperçu
print(info_SIRET.shape)
info_SIRET.head(3)

(32388, 18)
CPU times: user 1min 11s, sys: 2.56 s, total: 1min 14s
Wall time: 1min 13s


,SIRET,Date_inspection,Numero_inspection,siret,dateCreationEtablissement,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,dateDebut,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char
0,50846842800034,2024-02-05 00:00:00+00:00,23-110093-1,50846842800034,2010-12-02,11,NaN,2023-11-30T10:23:01,True,2.0,NaN,33039,NaN,2011-09-05,56.10A,O,na,33
1,90472398800013,2023-07-17 00:00:00+00:00,23-056603-1,90472398800013,2021-10-12,NaN,NaN,2022-09-03T14:40:36,True,2.0,NaN,93077,NaN,2022-06-02,47.11B,O,na,93
2,48792443300013,2023-03-08 00:00:00+00:00,23-018114-1,48792443300013,2005-12-24,01,NaN,2023-11-30T10:27:40,True,3.0,NaN,2B148,NaN,2008-01-01,55.10Z,O,na,2B


In [15]:
info_SIRET.describe()

,nombrePeriodesEtablissement
count,31260.000000
mean,2.600960
std,1.471423
min,1.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,16.000000


In [16]:
# Free memory
del SIRET
gc.collect()

0

___
___
# **Feature Engineering**

___
### **Date features**

In [17]:
# Conversion des dates
for k in ['Date_inspection', 'dateDebut', 'dateCreationEtablissement'] :
    info_SIRET[k] = pd.to_datetime(info_SIRET[k], utc=True)

# Différence de date
info_SIRET['Diffdate_days_DateInspection_dateDebut'] = (info_SIRET['Date_inspection'] - info_SIRET['dateDebut']) / np.timedelta64(1, 'D')
info_SIRET['Diffdate_days_DateInspection_dateCreationEtablissement'] = (info_SIRET['Date_inspection'] - info_SIRET['dateCreationEtablissement']) / np.timedelta64(1, 'D')

# Extract features
for k in ['dateDebut', 'dateCreationEtablissement'] :
    info_SIRET[f'year_{k}']  = info_SIRET[k].dt.year.fillna(-1).astype(int)
    info_SIRET[f'month_{k}'] = info_SIRET[k].dt.month.fillna(-1).astype(int)

___
### **Encoding**

In [18]:
def create_dico(k, df=info_SIRET, encode_most_frequent_only=True) :
    N = len(df)
    dico = df[k].value_counts() / N
    if encode_most_frequent_only :
        dico = dico[dico > 0.01]
    dico = {k:i for i, k in enumerate(dico.index)}
    return dico, {v:k for k, v in dico.items()}

In [19]:
info_SIRET['Code_NAF_1'] = info_SIRET['activitePrincipaleEtablissement'].astype(str).apply(lambda x : x[:2])
info_SIRET['Code_NAF_2'] = info_SIRET['activitePrincipaleRegistreMetiersEtablissement'].astype(str).apply(lambda x : x[:2])
info_SIRET['Egalite_NAF'] = (info_SIRET['Code_NAF_1'] == info_SIRET['Code_NAF_2']).astype(int)


dico_activitePrincipale, dico_activitePrincipale_INVERSE = create_dico('activitePrincipaleEtablissement')
dico_activitePrincipale2, dico_activitePrincipale2_INVERSE = create_dico('activitePrincipaleRegistreMetiersEtablissement')
dico_NAF1, dico_NAF1_INVERSE = create_dico('Code_NAF_1')
dico_NAF2, dico_NAF2_INVERSE = create_dico('Code_NAF_2')
dico_Effectifs, dico_Effectifs_INVERSE = create_dico('trancheEffectifsEtablissement')
dico_caractereEmployeur, dico_caractereEmployeur_INVERSE = create_dico('caractereEmployeurEtablissement')
dico_Siege, dico_Siege_INVERSE = create_dico('etablissementSiege')

dico_repetition, dico_repetition_INVERSE = create_dico('indiceRepetitionEtablissement')

dico_cedex, dico_cedex_INVERSE = create_dico('codeCedexEtablissement')
dico_cedex_2char, dico_cedex_2char_INVERSE = create_dico('codeCedexEtablissement_2char')
dico_commune, dico_commune_INVERSE = create_dico('codeCommuneEtablissement')
dico_commune_2char, dico_commune_2char_INVERSE = create_dico('codeCommuneEtablissement_2char')

dico_caractereEmployeur

{'O': 0, 'N': 1}

In [20]:
def encode_with_dict(df, col, dico) :
    return df[col].apply(lambda x : dico[x] if x in dico else len(dico))

info_SIRET['activitePrincipaleEtablissement'] = encode_with_dict(info_SIRET, 'activitePrincipaleEtablissement', dico_activitePrincipale)
info_SIRET['activitePrincipaleRegistreMetiersEtablissement'] = encode_with_dict(info_SIRET, 'activitePrincipaleRegistreMetiersEtablissement', dico_activitePrincipale2)
info_SIRET['Code_NAF_1'] = encode_with_dict(info_SIRET, 'Code_NAF_1', dico_NAF1)
info_SIRET['Code_NAF_2'] = encode_with_dict(info_SIRET, 'Code_NAF_2', dico_NAF2)
info_SIRET['trancheEffectifsEtablissement'] = encode_with_dict(info_SIRET, 'trancheEffectifsEtablissement', dico_Effectifs)
info_SIRET['caractereEmployeurEtablissement'] = encode_with_dict(info_SIRET, 'caractereEmployeurEtablissement', dico_caractereEmployeur)
info_SIRET['etablissementSiege'] = encode_with_dict(info_SIRET, 'etablissementSiege', dico_Siege)

info_SIRET['indiceRepetitionEtablissement'] = encode_with_dict(info_SIRET, 'indiceRepetitionEtablissement', dico_repetition)
info_SIRET['codeCedexEtablissement'] = encode_with_dict(info_SIRET, 'codeCedexEtablissement', dico_cedex)
info_SIRET['codeCedexEtablissement_2char'] = encode_with_dict(info_SIRET, 'codeCedexEtablissement_2char', dico_cedex_2char)
info_SIRET['codeCommuneEtablissement'] = encode_with_dict(info_SIRET, 'codeCommuneEtablissement', dico_commune)
info_SIRET['codeCommuneEtablissement_2char'] = encode_with_dict(info_SIRET, 'codeCommuneEtablissement_2char', dico_commune_2char)

# Aperçu
print(info_SIRET.shape)
info_SIRET.head(3)

(32388, 27)


,SIRET,Date_inspection,Numero_inspection,siret,dateCreationEtablissement,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,dateDebut,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF
0,50846842800034,2024-02-05 00:00:00+00:00,23-110093-1,50846842800034,2010-12-02 00:00:00+00:00,1,3,2023-11-30T10:23:01,0,2.0,1,5,0,2011-09-05 00:00:00+00:00,0,0,0,10,4536.0,4813.0,2011,9,2010,12,0,0,0
1,90472398800013,2023-07-17 00:00:00+00:00,23-056603-1,90472398800013,2021-10-12 00:00:00+00:00,10,3,2022-09-03T14:40:36,0,2.0,1,5,0,2022-06-02 00:00:00+00:00,16,0,0,4,410.0,643.0,2022,6,2021,10,2,0,0
2,48792443300013,2023-03-08 00:00:00+00:00,23-018114-1,48792443300013,2005-12-24 00:00:00+00:00,3,3,2023-11-30T10:27:40,0,3.0,1,5,0,2008-01-01 00:00:00+00:00,6,0,0,16,5545.0,6283.0,2008,1,2005,12,8,0,0


___
## **Selection de colonnes**

In [21]:
info_SIRET = info_SIRET.drop(columns = ['siret', 'dateCreationEtablissement', 'dateDernierTraitementEtablissement', 'dateDebut'])

___
___
# **Sauvegarde**

In [22]:
# Suppression des doublons
info_SIRET = info_SIRET.sort_values(by = ["Numero_inspection", "Code_NAF_1", "Code_NAF_2"],
                                    ascending = True,
                                   ).reset_index(drop=True)
info_SIRET['n_priority'] = info_SIRET.groupby(["Numero_inspection"]).transform("cumcount")
info_SIRET = info_SIRET[info_SIRET['n_priority'] == 0].drop(columns = ["n_priority"]).reset_index(drop=True)

print(info_SIRET.shape)
info_SIRET.head()

(32388, 23)


,SIRET,Date_inspection,Numero_inspection,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF
0,83214138600017,2023-10-12 00:00:00+00:00,16218678,10,3,2,NaN,1,5,0,26,2,1,25,NaN,NaN,-1,-1,-1,-1,6,0,1
1,83026676300010,2023-02-14 00:00:00+00:00,16728080,0,2,0,2.0,1,5,0,10,0,0,25,2054.0,2071.0,2017,7,2017,6,1,1,1
2,89787374100010,2023-02-14 00:00:00+00:00,16728383,10,0,0,2.0,1,5,0,1,1,0,25,-269.0,683.0,2023,11,2021,4,0,2,1
3,85384956000018,2023-02-14 00:00:00+00:00,16728824,0,3,0,2.0,1,5,0,1,0,0,9,1230.0,1252.0,2019,10,2019,9,0,0,0
4,83358851000036,2023-02-15 00:00:00+00:00,16729489,4,3,0,2.0,1,5,0,26,0,0,9,1750.0,44970.0,2018,5,1900,1,1,1,1


In [23]:
info_SIRET["Numero_inspection"].value_counts()

Numero_inspection
16218678       1
23-077176-1    1
23-077207-1    1
23-077205-1    1
23-077204-1    1
              ..
23-037312-1    1
23-037304-1    1
23-037303-2    1
23-037303-1    1
24-009286-1    1
Name: count, Length: 32388, dtype: int64

In [24]:
info_SIRET.to_csv('info_SIRET.csv', index=False)